In [1]:
import numpy as np

In [2]:
import torch
from torch.autograd import Variable
import numpy
import unittest

**Module** is an abstract class which defines fundamental methods necessary for a training a neural network. You do not need to change anything here, just read the comments.

In [3]:
class Module(object):
    """
    Basically, you can think of a module as of a something (black box) 
    which can process `input` data and produce `ouput` data.
    This is like applying a function which is called `forward`: 
        
        output = module.forward(input)
    
    The module should be able to perform a backward pass: to differentiate the `forward` function. 
    More, it should be able to differentiate it if is a part of chain (chain rule).
    The latter implies there is a gradient from previous step of a chain rule. 
    
        gradInput = module.backward(input, gradOutput)
    """
    def __init__ (self):
        self.output = None
        self.gradInput = None
        self.training = True
    
    def forward(self, input):
        """
        Takes an input object, and computes the corresponding output of the module.
        """
        return self.updateOutput(input)

    def backward(self,input, gradOutput):
        """
        Performs a backpropagation step through the module, with respect to the given input.
        
        This includes 
         - computing a gradient w.r.t. `input` (is needed for further backprop),
         - computing a gradient w.r.t. parameters (to update parameters while optimizing).
        """
        self.updateGradInput(input, gradOutput)
        self.accGradParameters(input, gradOutput)
        return self.gradInput
    

    def updateOutput(self, input):
        """
        Computes the output using the current parameter set of the class and input.
        This function returns the result which is stored in the `output` field.
        
        Make sure to both store the data in `output` field and return it. 
        """
        
        # The easiest case:
            
        # self.output = input 
        # return self.output
        
        pass

    def updateGradInput(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own input. 
        This is returned in `gradInput`. Also, the `gradInput` state variable is updated accordingly.
        
        The shape of `gradInput` is always the same as the shape of `input`.
        
        Make sure to both store the gradients in `gradInput` field and return it.
        """
        
        # The easiest case:
        
        # self.gradInput = gradOutput 
        # return self.gradInput
        
        pass   
    
    def accGradParameters(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own parameters.
        No need to override if module has no parameters (e.g. ReLU).
        """
        pass
    
    def zeroGradParameters(self): 
        """
        Zeroes `gradParams` variable if the module has params.
        """
        pass
        
    def getParameters(self):
        """
        Returns a list with its parameters. 
        If the module does not have parameters return empty list. 
        """
        return []
        
    def getGradParameters(self):
        """
        Returns a list with gradients with respect to its parameters. 
        If the module does not have parameters return empty list. 
        """
        return []
    
    def train(self):
        """
        Sets training mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = True
    
    def evaluate(self):
        """
        Sets evaluation mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = False
    
    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want 
        to have readable description. 
        """
        return "Module"

# Sequential container

**Define** a forward and backward pass procedures.

In [4]:
class Sequential(Module):
    """
         This class implements a container, which processes `input` data sequentially. 
         
         `input` is processed by each module (layer) in self.modules consecutively.
         The resulting array is called `output`. 
    """
    
    def __init__ (self):
        super(Sequential, self).__init__()
        self.modules = []
   
    def add(self, module):
        """
        Adds a module to the container.
        """
        self.modules.append(module)

    def updateOutput(self, input):
        """
        Basic workflow of FORWARD PASS:
        
            y_0    = module[0].forward(input)
            y_1    = module[1].forward(y_0)
            ...
            output = module[n-1].forward(y_{n-2})   
            
            
        Just write a little loop. 
        """

        current_input = input # инициализирую переменную
        for module in self.modules: # пробегаюсь циклом по всем input`ам
            current_input = module.forward(current_input) # применяю метод forward для input`а
        self.output = current_input # перезаписываю input в цикле
        return self.output # выхожу из цикла

    def backward(self, input, gradOutput):
        """
        Workflow of BACKWARD PASS:
            
            g_{n-1} = module[n-1].backward(y_{n-2}, gradOutput)
            g_{n-2} = module[n-2].backward(y_{n-3}, g_{n-1})
            ...
            g_1 = module[1].backward(y_0, g_2)   
            gradInput = module[0].backward(input, g_1)   
             
             
        !!!
                
        To ech module you need to provide the input, module saw while forward pass, 
        it is used while computing gradients. 
        Make sure that the input for `i-th` layer the output of `module[i]` (just the same input as in forward pass) 
        and NOT `input` to this Sequential module. 
        
        !!!
        
        """
        current_gradOutput = gradOutput # задаю начальный градиент
        gradInputs = [] #  список для градиентов входов 
        for i in range(len(self.modules) - 1, -1, -1): # иду по модулю с конца, это же backward:)
            module = self.modules[i] # получаю данные с текущего слоя
            if i == 0:
                gradInput = module.backward(input, current_gradOutput) # на 1 шагу использую backward с исходным входом и текущим градиентом
            else: # для последующих параметров
                gradInput = module.backward(self.modules[i-1].output, current_gradOutput) # использую обратный проход с выходом предыдущего модуля и текущим градиентом
            current_gradOutput = gradInput # обновляю градиент
            gradInputs.insert(0, gradInput) # добавляю градиент в свой список для градиентов
        self.gradInput = gradInputs[0]
        return self.gradInput # возвращаю градиент для последовательности, начинаю цикл опять и опять
      

    def zeroGradParameters(self): 
        for module in self.modules:
            module.zeroGradParameters()
    
    def getParameters(self):
        """
        Should gather all parameters in a list.
        """
        return [x.getParameters() for x in self.modules]
    
    def getGradParameters(self):
        """
        Should gather all gradients w.r.t parameters in a list.
        """
        return [x.getGradParameters() for x in self.modules]
    
    def __repr__(self):
        string = "".join([str(x) + '\n' for x in self.modules])
        return string
    
    def __getitem__(self,x):
        return self.modules.__getitem__(x)
    
    def train(self):
        """
        Propagates training parameter through all modules
        """
        self.training = True
        for module in self.modules:
            module.train()
    
    def evaluate(self):
        """
        Propagates training parameter through all modules
        """
        self.training = False
        for module in self.modules:
            module.evaluate()

# Layers

## 1. Linear transform layer
Also known as dense layer, fully-connected layer, FC-layer, InnerProductLayer (in caffe), affine transform
- input:   **`batch_size x n_feats1`**
- output: **`batch_size x n_feats2`**

In [5]:
class Linear(Module):
    """
    A module which applies a linear transformation 
    A common name is fully-connected layer, InnerProductLayer in caffe. 
    
    The module should work with 2D input of shape (n_samples, n_feature).
    """
    def __init__(self, n_in, n_out):
        super(Linear, self).__init__()
       
        # This is a nice initialization
        stdv = 1./np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size = (n_out, n_in))
        self.b = np.random.uniform(-stdv, stdv, size = n_out)
        
        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)
        
    def updateOutput(self, input):
        self.output = input @ self.W.T + self.b
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        
        self.gradInput = gradOutput @ self.W
        return self.gradInput
    
    def accGradParameters(self, input, gradOutput):
        # Your code goes here. ################################################
        self.gradW += gradOutput.T @ input 
        self.gradb += gradOutput.sum(axis=0)
    
    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)
        
    def getParameters(self):
        return [self.W, self.b]
    
    def getGradParameters(self):
        return [self.gradW, self.gradb]
    
    def __repr__(self):
        s = self.W.shape
        q = 'Linear %d -> %d' %(s[1],s[0])
        return q

In [6]:
layer = Linear(3, 2)
layer

Linear 3 -> 2

In [7]:
layer.W

array([[-0.39454568, -0.46158723,  0.31347043],
       [ 0.12818129, -0.32775629, -0.15234367]])

In [8]:
layer.output

In [9]:
batch_size, n_in, n_out = 2, 3, 4

# layers initialization
torch_layer = torch.nn.Linear(n_in, n_out)
custom_layer = Linear(n_in, n_out)
custom_layer.W = torch_layer.weight.data.numpy()
custom_layer.b = torch_layer.bias.data.numpy()

layer_input = np.random.uniform(-10, 10, (batch_size, n_in)).astype(np.float32)
next_layer_grad = np.random.uniform(-10, 10, (batch_size, n_out)).astype(np.float32)

In [10]:
layer_input

array([[-3.5511076, -5.157596 , -4.052141 ],
       [ 7.9145117, -6.646369 , -9.822641 ]], dtype=float32)

In [11]:
next_layer_grad

array([[-9.908463 , -7.52101  , -7.8581767,  9.72647  ],
       [-9.672577 , -3.6494982,  7.615905 ,  8.844072 ]], dtype=float32)

In [12]:
layer_input # [batch_size x n_in]

array([[-3.5511076, -5.157596 , -4.052141 ],
       [ 7.9145117, -6.646369 , -9.822641 ]], dtype=float32)

In [13]:
custom_layer.W # [n_out x n_in]

array([[ 0.33817244, -0.12206075, -0.52343965],
       [-0.2787326 ,  0.47177863,  0.44343758],
       [-0.31535113, -0.5492385 ,  0.42176247],
       [-0.31061807, -0.04576802,  0.30938226]], dtype=float32)

In [14]:
layer_input @ custom_layer.W.T + custom_layer.b

array([[ 1.236616 , -3.3812633,  1.7272439, -0.1624274],
       [ 8.316202 , -9.838333 , -3.504541 , -5.4410086]], dtype=float32)

In [15]:
# 1. check layer output
custom_layer_output = custom_layer.updateOutput(layer_input)
layer_input_var = Variable(torch.from_numpy(layer_input), requires_grad=True)
torch_layer_output_var = torch_layer(layer_input_var)
np.allclose(torch_layer_output_var.data.numpy(), custom_layer_output, atol=1e-6)

True

In [16]:
# 2. check layer input grad
custom_layer_grad = custom_layer.updateGradInput(layer_input, next_layer_grad)
torch_layer_output_var.backward(torch.from_numpy(next_layer_grad))
torch_layer_grad_var = layer_input_var.grad
np.allclose(torch_layer_grad_var.data.numpy(), custom_layer_grad, atol=1e-6)

True

In [17]:
# 3. check layer parameters grad
custom_layer.accGradParameters(layer_input, next_layer_grad)
weight_grad = custom_layer.gradW
bias_grad = custom_layer.gradb
torch_weight_grad = torch_layer.weight.grad.data.numpy()
torch_bias_grad = torch_layer.bias.grad.data.numpy()

In [18]:
np.allclose(torch_weight_grad, weight_grad, atol=1e-6)

True

In [19]:
np.allclose(torch_bias_grad, bias_grad, atol=1e-6)

True

## 2. SoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{softmax}(x)_i = \frac{\exp x_i} {\sum_j \exp x_j}$

Recall that $\text{softmax}(x) == \text{softmax}(x - \text{const})$. It makes possible to avoid computing exp() from large argument.

In [20]:
class SoftMax(Module):
    def __init__(self):
         super(SoftMax, self).__init__()
    
    def updateOutput(self, input):
        # start with normalization for numerical stability
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))
        exps = np.exp(self.output) # получаю экспоненту к входным данным
        self.output = exps / np.sum(exps, axis=1, keepdims=True) # вычисляю значение софтмакса лейбла как отношение экспоненты этой метки к сумме экспонент => получаю степень уверенности в классе
        return self.output 
    
    def updateGradInput(self, input, gradOutput):
        softmax_output = self.output # инициализация softmax_output
        self.gradInput = np.empty_like(gradOutput) # инициализация массива градиентов size массива = size gradOutput
        for i in range(len(gradOutput)):
            y = softmax_output[i] # cофтмакс выход для i-го элемента
            dy = gradOutput[i] #uрадиент функции потерь по выходу софтмакса для i-го элемента
            self.gradInput[i] = y * (dy - np.dot(dy, y)) # считаю градиент софтмакса (вектор наибольшего роста функции)
        return self.gradInput
    
    def __repr__(self):
        return "SoftMax"    

## 3. LogSoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{logsoftmax}(x)_i = \log\text{softmax}(x)_i = x_i - \log {\sum_j \exp x_j}$

The main goal of this layer is to be used in computation of log-likelihood loss.

In [21]:
class LogSoftMax(Module):
    def __init__(self):
         super(LogSoftMax, self).__init__()
    
    def updateOutput(self, input):
        # start with normalization for numerical stability 
        self.output = np.subtract(input, input.max(axis=1, keepdims=True)) # нормализация
        # считаю логарифм суммы экспонент входных данных после нормализации
        # и вычитаю логарифм суммы из нормализованных входных данных для получения log-softmax
        self.output -= np.log(np.sum(np.exp(self.output), axis=1, keepdims=True)) #
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        softmax_output = np.exp(self.output)# считаю софтмакс по логсофмтаксу - использую обратную функцию к логарифму: экспоненту
        # считаю градиент логсофтмакса, output - степень уверенности классификатора
        self.gradInput = gradOutput - softmax_output * np.sum(gradOutput, axis=1, keepdims=True)
        return self.gradInput
    
    def __repr__(self):
        return "LogSoftMax"

## 4. Batch normalization
One of the most significant recent ideas that impacted NNs a lot is [**Batch normalization**](http://arxiv.org/abs/1502.03167). The idea is simple, yet effective: the features should be whitened ($mean = 0$, $std = 1$) all the way through NN. This improves the convergence for deep models letting it train them for days but not weeks. **You are** to implement the first part of the layer: features normalization. The second part (`ChannelwiseScaling` layer) is implemented below.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

The layer should work as follows. While training (`self.training == True`) it transforms input as $$y = \frac{x - \mu}  {\sqrt{\sigma + \epsilon}}$$
where $\mu$ and $\sigma$ - mean and variance of feature values in **batch** and $\epsilon$ is just a small number for numericall stability. Also during training, layer should maintain exponential moving average values for mean and variance: 
```
    self.moving_mean = self.moving_mean * alpha + batch_mean * (1 - alpha)
    self.moving_variance = self.moving_variance * alpha + batch_variance * (1 - alpha)
```
During testing (`self.training == False`) the layer normalizes input using moving_mean and moving_variance. 

Note that decomposition of batch normalization on normalization itself and channelwise scaling here is just a common **implementation** choice. In general "batch normalization" always assumes normalization + scaling.

In [22]:
## В целом, батч нормализация - это просто скалеры из ML, для которых выделен отдельный слой, в нашем слуаче это просто 
## StandardScaler - мы отнимаем мат ожидание и делим на стандартное отклонение, чтобы СВ имела нормальное распределение с 
## Мат ожиданием 0 и дисперсией 1 - это позволит работать нашей нейронке с одинаково масштабными данными, что позволяет
## абстрагироваться от размеров данных и сосредоточиться на их информативности
## в знаменателе стоит эпсилон поскольку дисперсия >= 0 (0 если const), поэтому прибавляем что-то очень маленькое, чтобы избежать
## деления на 0, кстати, в формуле как будто ошибка, в знаменателе должна быть сигма квадрат
class BatchNormalization(Module): ## создаю класс БатчНормализации
    EPS = 1e-3
    def __init__(self, alpha = 0.):
        super(BatchNormalization, self).__init__()
        self.alpha = alpha # коэфф скользящего среднего для eps
        self.moving_mean = None  # коэф скользящего среднего для М.O
        self.moving_variance = None # коэф скользящего среднего для дисперсии
        # Все эти скользяшие средние нужны для нормализации батча
    def updateOutput(self, input):
        if self.training:
            batch_mean = np.mean(input, axis=0) #среднее по батчу
            batch_variance = np.var(input, axis=0) #дисперсия по батчу
            
            self.output = (input - batch_mean) / np.sqrt(batch_variance + self.EPS) #отнормализовали
            # обновляем параметры, если они есть
            if self.moving_mean is None:
                self.moving_mean = batch_mean
                self.moving_variance = batch_variance
            else:
                self.moving_mean = self.moving_mean * self.alpha + batch_mean * (1 - self.alpha)
                self.moving_variance = self.moving_variance * self.alpha + batch_variance * (1 - self.alpha)
        else:
            # это для валидации
            self.output = (input - self.moving_mean) / np.sqrt(self.moving_variance + self.EPS)
        
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        if self.training:
            # Считаю всё для градиентов
            batch_mean = np.mean(input, axis=0)
            batch_variance = np.var(input, axis=0)
            m = input.shape[0]
            X_mu = input - batch_mean
            std_inv = 1. / np.sqrt(batch_variance + self.EPS)
            # производные
            dX_norm = gradOutput * std_inv
            dvar = np.sum(gradOutput * X_mu, axis=0) * -.5 * std_inv**3
            dmu = np.sum(gradOutput * -std_inv, axis=0) + dvar * np.mean(-2. * X_mu, axis=0)
            # градиент
            self.gradInput = dX_norm + dvar * 2 * X_mu / m + dmu / m
        else:
            # мастшатибуремся обратно для валидации, чтобы правильно интерпритировать результаты работы сетки
            self.gradInput = gradOutput / np.sqrt(self.moving_variance + self.EPS)
        
        return self.gradInput
    
    def __repr__(self):
        return "BatchNormalization"
    # Режим обучения
    def train(self):
        self.training = True
    # Режим валидации
    def evaluate(self):
        self.training = False

In [23]:
class ChannelwiseScaling(Module):
    """
       Implements linear transform of input y = \gamma * x + \beta
       where \gamma, \beta - learnable vectors of length x.shape[-1]
    """
    def __init__(self, n_out):
        super(ChannelwiseScaling, self).__init__()

        stdv = 1./np.sqrt(n_out)
        self.gamma = np.random.uniform(-stdv, stdv, size=n_out)
        self.beta = np.random.uniform(-stdv, stdv, size=n_out)
        
        self.gradGamma = np.zeros_like(self.gamma)
        self.gradBeta = np.zeros_like(self.beta)

    def updateOutput(self, input):
        self.output = input * self.gamma + self.beta
        return self.output
        
    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput * self.gamma
        return self.gradInput
    
    def accGradParameters(self, input, gradOutput):
        self.gradBeta = np.sum(gradOutput, axis=0)
        self.gradGamma = np.sum(gradOutput*input, axis=0)
    
    def zeroGradParameters(self):
        self.gradGamma.fill(0)
        self.gradBeta.fill(0)
        
    def getParameters(self):
        return [self.gamma, self.beta]
    
    def getGradParameters(self):
        return [self.gradGamma, self.gradBeta]
    
    def __repr__(self):
        return "ChannelwiseScaling"

Practical notes. If BatchNormalization is placed after a linear transformation layer (including dense layer, convolutions, channelwise scaling) that implements function like `y = weight * x + bias`, than bias adding become useless and could be omitted since its effect will be discarded while batch mean subtraction. If BatchNormalization (followed by `ChannelwiseScaling`) is placed before a layer that propagates scale (including ReLU, LeakyReLU) followed by any linear transformation layer than parameter `gamma` in `ChannelwiseScaling` could be freezed since it could be absorbed into the linear transformation layer.

## 5. Dropout
Implement [**dropout**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). The idea and implementation is really simple: just multimply the input by $Bernoulli(p)$ mask. Here $p$ is probability of an element to be zeroed.

This has proven to be an effective technique for regularization and preventing the co-adaptation of neurons.

While training (`self.training == True`) it should sample a mask on each iteration (for every batch), zero out elements and multiply elements by $1 / (1 - p)$. The latter is needed for keeping mean values of features close to mean values which will be in test mode. When testing this module should implement identity transform i.e. `self.output = input`.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [24]:
class Dropout(Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()
        self.p = p  # с этой вероятностью выключаем нейрон
        
    def updateOutput(self, input):
        if self.training:
            self.mask = np.random.binomial(1, 1 - self.p, size=input.shape) #  генерирую нужное распределение
            self.output = input * self.mask / (1 - self.p) # применяю маску с нужным распределением к инпутам
        else:
            # при тестиорвании дропают не используем
            self.output = input
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # считаем градиенты, для трейна используем нашу маску
        if self.training:
            self.gradInput = gradOutput * self.mask / (1 - self.p)
            # на тесте маска не нужна, все активны
        else:
            self.gradInput = gradOutput
        return self.gradInput
    
    def __repr__(self):
        return "Dropout"

# Activation functions

Here's the complete example for the **Rectified Linear Unit** non-linearity (aka **ReLU**): 

In [25]:
class ReLU(Module):
    def __init__(self):
         super(ReLU, self).__init__()
    
    def updateOutput(self, input):
        self.output = np.maximum(input, 0)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.multiply(gradOutput , input > 0)
        return self.gradInput
    
    def __repr__(self):
        return "ReLU"

## 6. Leaky ReLU
Implement [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs). Expriment with slope. 

<img src = 'Selection_019.png'/>

In [26]:
class LeakyReLU(Module):
    def __init__(self, slope=0.03):
        super(LeakyReLU, self).__init__()
        self.slope = slope  # Коэффициент наклона для отрицательной части 
    
    def updateOutput(self, input):
        # применяю LeakyReLU к инпутам,  для положительных значений функция возвращает само значение
        # для отрицательных значений функция возвращает значение, умноженное на коэффициент slope
        # что позволяет избежать "умирающих" нейронов, как в случае с обычным ReLU.
        self.output = np.where(input > 0, input, self.slope * input)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # считаю градиент функции активации, для положительных значений входа
        # градиент функции потерь передается без изменений. Для отрицательных значений
        # градиент умножается на коэффициент slope. Что позволяет считать градиент
        # даже по выключенным нейроны, предотвращая их полное затухание
        self.gradInput = np.where(input > 0, gradOutput, self.slope * gradOutput)
        return self.gradInput
    
    def __repr__(self):
        return "LeakyReLU"

## 7. ELU
Implement [**Exponential Linear Units**](http://arxiv.org/abs/1511.07289) activations.

<img src = 'i.webp'/>

In [27]:
class ELU(Module):
    def __init__(self, alpha=1.0):
        super(ELU, self).__init__()
        self.alpha = alpha # Коэффициент масштабирования для отрицательных входов.
        
    def updateOutput(self, input):
        # применению  ELU к входным данным: для положительных входов функция возвращает
        # значение входа,  для отрицательных входов возвращает alpha * (exp(input) - 1)
        self.output = np.maximum(0, input) + self.alpha * (np.exp(np.minimum(0, input)) - 1)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        # считаю производную ELU: для положительных входов градиент остается неизменным (равен 1),
        # а для отрицательных входов градиент равен alpha * exp(input), что соответствует производной
        # ELU для отрицательных значений
        grad = self.alpha * np.exp(np.minimum(0, input))
        grad[input > 0] = 1
        self.gradInput = gradOutput * grad
        return self.gradInput
    
    def __repr__(self):
        return "ELU"

## 8. SoftPlus
Implement [**SoftPlus**](https://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29) activations. Look, how they look a lot like ReLU.

## Здесь лежат все функции активации

<img src = '2024-04-07_13-49-51.png'/>

In [28]:
class SoftPlus(Module):
    def __init__(self):
        super(SoftPlus, self).__init__()
    # SoftPlus инициализация
    def updateOutput(self, input):
        self.output = np.log(1 + np.exp(input))
        return self.output
    ## считаю градиент
    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput * (1 / (1 + np.exp(-input)))
        return self.gradInput
    
    def __repr__(self):
        return "SoftPlus"

# Criterions

Criterions are used to score the models answers. 

In [29]:
class Criterion(object):
    def __init__ (self):
        self.output = None
        self.gradInput = None
        
    def forward(self, input, target):
        """
            Given an input and a target, compute the loss function 
            associated to the criterion and return the result.
            
            For consistency this function should not be overrided,
            all the code goes in `updateOutput`.
        """
        return self.updateOutput(input, target)

    def backward(self, input, target):
        """
            Given an input and a target, compute the gradients of the loss function
            associated to the criterion and return the result. 

            For consistency this function should not be overrided,
            all the code goes in `updateGradInput`.
        """
        return self.updateGradInput(input, target)
    
    def updateOutput(self, input, target):
        """
        Function to override.
        """
        return self.output

    def updateGradInput(self, input, target):
        """
        Function to override.
        """
        return self.gradInput   

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want 
        to have readable description. 
        """
        return "Criterion"

The **MSECriterion**, which is basic L2 norm usually used for regression, is implemented here for you.
- input:   **`batch_size x n_feats`**
- target: **`batch_size x n_feats`**
- output: **scalar**

In [30]:
class MSECriterion(Criterion):
    def __init__(self):
        super(MSECriterion, self).__init__()
        
    def updateOutput(self, input, target):   
        self.output = np.sum(np.power(input - target,2)) / input.shape[0]
        return self.output 
 
    def updateGradInput(self, input, target):
        self.gradInput  = (input - target) * 2 / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "MSECriterion"

## 9. Negative LogLikelihood criterion (numerically unstable)
You task is to implement the **ClassNLLCriterion**. It should implement [multiclass log loss](http://scikit-learn.org/stable/modules/model_evaluation.html#log-loss). Nevertheless there is a sum over `y` (target) in that formula, 
remember that targets are one-hot encoded. This fact simplifies the computations a lot. Note, that criterions are the only places, where you divide by batch size. Also there is a small hack with adding small number to probabilities to avoid computing log(0).
- input:   **`batch_size x n_feats`** - probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**



In [31]:
class ClassNLLCriterionUnstable(Criterion):
    EPS = 1e-15
    def __init__(self):
        a = super(ClassNLLCriterionUnstable, self)
        super(ClassNLLCriterionUnstable, self).__init__()
        
    def updateOutput(self, input, target): 
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)
        # вычисление отрицательного логарифмического правдоподобия (Negative LogLikelihood)
        #умноженаю целевых значений на логарифм входных данных, с последующим усреднением по размеру батча.
        self.output = -np.sum(target * np.log(input_clamp)) / input.shape[0]
        return self.output

    def updateGradInput(self, input, target):
        
        # Use this trick to avoid numerical errors
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)
        # считаю градиент функции потерь по входным данным, усредняю по размеру батча
        self.gradInput = -(target / input_clamp) / input.shape[0]
        return self.gradInput
    
    def __repr__(self):
        return "ClassNLLCriterionUnstable"

## 10. Negative LogLikelihood criterion (numerically stable)
- input:   **`batch_size x n_feats`** - log probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**

Task is similar to the previous one, but now the criterion input is the output of log-softmax layer. This decomposition allows us to avoid problems with computation of forward and backward of log().

In [32]:
class ClassNLLCriterion(Criterion):
    def __init__(self):
        a = super(ClassNLLCriterion, self)
        super(ClassNLLCriterion, self).__init__()
        
    def updateOutput(self, input, target): 
        # считаю значение функции потерь(NLL), инпут уже содержит логарифмы вероятностей - это и есть главное отличие от предыдущей задачки
        #(степени уврененности классификатора) классов
        # умножаю таргеты на инпуты, суммирую и получаю суммарную потерю
        # делю на количество примеров => нормализую потерю, делая ее усредненной по батчу
        self.output = -np.sum(target * input) / input.shape[0]
        return self.output

    def updateGradInput(self, input, target):
        # считаю градиент функции потерь по входным данным, усредняю по размеру батча
        self.gradInput = -target / input.shape[0]
        return self.gradInput
    
    def __repr__(self):
        return "ClassNLLCriterion"